In [1]:
import pandas as pd
import numpy as np
import os

# 工作路径改为/quoraDupli
os.chdir(os.path.dirname(os.getcwd()))
print os.getcwd()

/software/home/chenzh/software/jupyter/quoraDupli


In [2]:
# 载入经过预处理的数据
df = pd.read_pickle(os.path.join('data', 'questionVector.pickle'))

In [16]:
from sklearn.model_selection import KFold
from keras.optimizers import Adam
# 将dataframe打乱顺序
df = df.reindex(np.random.permutation(df.index))

# 初始化
num_df = df.shape[0]
X = np.zeros([num_df, 2, 300])  # (404288, 2, 300)
Y = np.zeros([num_df])

b = [a[None,:] for a in list(df['q1Vector'].values)]
q1_feats = np.concatenate(b, axis=0)  # (404288, 300)

b = [a[None,:] for a in list(df['q2Vector'].values)]
q2_feats = np.concatenate(b, axis=0)

X[:,0,:] = q1_feats
X[:,1,:] = q2_feats
Y = df['is_duplicate'].values

del b
del q1_feats
del q2_feats

f1_list = []
kf = KFold(n_splits=10)
    cv_num = 1
for train_index, test_index in kf.split(X):
    print '第%d次交叉验证开始... ...' % cv_num
    X_train = X[train_index]
    X_test = X[test_index]
    Y_train = Y[train_index]
    Y_test = Y[test_index]
    
    # 特征向量规范
    X_train_norm = np.zeros_like(X_train)
    d = (np.sum(X_train[:,0,:] ** 2, 1) ** (0.5))
    X_train_norm[:,0,:] = (X_train[:,0,:].T / (d + 1e-8)).T
    d = (np.sum(X_train[:,1,:] ** 2, 1) ** (0.5))
    X_train_norm[:,1,:] = (X_train[:,1,:].T / (d + 1e-8)).T
    
    X_test_norm = np.zeros_like(X_test)
    d = (np.sum(X_test[:,0,:] ** 2, 1) ** (0.5))
    X_test_norm[:,0,:] = (X_test[:,0,:].T / (d + 1e-8)).T
    d = (np.sum(X_test[:,1,:] ** 2, 1) ** (0.5))
    X_test_norm[:,1,:] = (X_test[:,1,:].T / (d + 1e-8)).T
    
    net = create_network(300)
    optimizer = Adam(lr=0.001)
    net.compile(loss=contrastive_loss, optimizer=optimizer)
    for epoch in range(50):
        net.fit([X_train_norm[:,0,:], X_train_norm[:,1,:]], Y_train,
            validation_data=([X_test_norm[:,0,:], X_test_norm[:,1,:]], Y_test),
            batch_size=128, nb_epoch=1, shuffle=True, )
        # 预测
        pred = net.predict([X_test_norm[:,0,:], X_test_norm[:,1,:]], batch_size=128)
        te_acc = compute_accuracy(pred, Y_test)
        te_result = compute_result(pred, Y_test)
        print '测试集上正确率: %0.2f%%' % (100 * te_result['accuracy'])
        print '测试集上准确率: %0.2f%%' % (100 * te_result['precision'])
        print '测试集上召回率: %0.2f%%' % (100 * te_result['recall'])
        print '测试集上F1值: %0.6f' % te_result['f1']
    f1_list.append(te_result['f1'])
    print '第%d次交叉验证完成。' % cv_num
    cv_num += 1
print '十次交叉验证F1值取平均：%0.6f' % np.array(f1_list).mean()

第1次交叉验证开始... ...


/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:37: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/software/home/chenzh/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:40: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ba..., inputs=Tensor("in...)`
/

Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 22s - loss: 5.3048 - val_loss: 0.2176
测试集上正确率: 70.02%
测试集上准确率: 57.87%
测试集上召回率: 70.12%
测试集上F1值: 0.634108
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1828 - val_loss: 0.1743
测试集上正确率: 74.12%
测试集上准确率: 63.27%
测试集上召回率: 71.87%
测试集上F1值: 0.672980
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1644 - val_loss: 0.1620
测试集上正确率: 76.61%
测试集上准确率: 67.85%
测试集上召回率: 70.06%
测试集上F1值: 0.689395
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1542 - val_loss: 0.1551
测试集上正确率: 77.84%
测试集上准确率: 70.04%
测试集上召回率: 70.23%
测试集上F1值: 0.701333
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1463 - val_loss: 0.1

363859/363859 [==============================] - 20s - loss: 0.1015 - val_loss: 0.1345
测试集上正确率: 82.07%
测试集上准确率: 74.92%
测试集上召回率: 77.58%
测试集上F1值: 0.762233
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1011 - val_loss: 0.1344
测试集上正确率: 82.19%
测试集上准确率: 75.37%
测试集上召回率: 77.14%
测试集上F1值: 0.762404
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1009 - val_loss: 0.1344
测试集上正确率: 82.12%
测试集上准确率: 75.87%
测试集上召回率: 75.88%
测试集上F1值: 0.758761
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1006 - val_loss: 0.1350
测试集上正确率: 82.00%
测试集上准确率: 75.46%
测试集上召回率: 76.20%
测试集上F1值: 0.758295
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1004 - val_loss: 0.1342
测试集上正确率: 81.94%
测试集上准确率: 74.50%
测试集上召回率: 77.95%
测试集上F1值: 

363859/363859 [==============================] - 20s - loss: 0.1072 - val_loss: 0.1331
测试集上正确率: 82.21%
测试集上准确率: 75.71%
测试集上召回率: 76.27%
测试集上F1值: 0.759882
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1066 - val_loss: 0.1340
测试集上正确率: 82.14%
测试集上准确率: 76.09%
测试集上召回率: 75.27%
测试集上F1值: 0.756764
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1063 - val_loss: 0.1339
测试集上正确率: 82.27%
测试集上准确率: 76.05%
测试集上召回率: 75.82%
测试集上F1值: 0.759348
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1058 - val_loss: 0.1334
测试集上正确率: 82.22%
测试集上准确率: 75.62%
测试集上召回率: 76.46%
测试集上F1值: 0.760381
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1052 - val_loss: 0.1334
测试集上正确率: 82.08%
测试集上准确率: 75.82%
测试集上召回率: 75.53%
测试集上F1值: 

...

363859/363859 [==============================] - 20s - loss: 0.1007 - val_loss: 0.1359
测试集上正确率: 81.76%
测试集上准确率: 74.31%
测试集上召回率: 76.86%
测试集上F1值: 0.755632
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1003 - val_loss: 0.1359
测试集上正确率: 81.90%
测试集上准确率: 74.82%
测试集上召回率: 76.38%
测试集上F1值: 0.755936
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1003 - val_loss: 0.1356
测试集上正确率: 81.98%
测试集上准确率: 74.93%
测试集上召回率: 76.47%
测试集上F1值: 0.756930
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1000 - val_loss: 0.1354
测试集上正确率: 82.08%
测试集上准确率: 74.97%
测试集上召回率: 76.80%
测试集上F1值: 0.758756
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.0996 - val_loss: 0.1359
测试集上正确率: 82.13%
测试集上准确率: 75.11%
测试集上召回率: 76.75%
测试集上F1值: 

363859/363859 [==============================] - 20s - loss: 0.1051 - val_loss: 0.1361
测试集上正确率: 81.61%
测试集上准确率: 74.09%
测试集上召回率: 76.11%
测试集上F1值: 0.750863
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1045 - val_loss: 0.1357
测试集上正确率: 81.64%
测试集上准确率: 74.07%
测试集上召回率: 76.29%
测试集上F1值: 0.751665
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1044 - val_loss: 0.1356
测试集上正确率: 81.81%
测试集上准确率: 74.25%
测试集上召回率: 76.62%
测试集上F1值: 0.754145
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1039 - val_loss: 0.1355
测试集上正确率: 81.81%
测试集上准确率: 74.11%
测试集上召回率: 76.90%
测试集上F1值: 0.754809
Train on 363859 samples, validate on 40429 samples
Epoch 1/1
363859/363859 [==============================] - 20s - loss: 0.1035 - val_loss: 0.1356
测试集上正确率: 81.70%
测试集上准确率: 73.84%
测试集上召回率: 77.04%
测试集上F1值: 

363860/363860 [==============================] - 20s - loss: 0.1115 - val_loss: 0.1347
测试集上正确率: 81.57%
测试集上准确率: 74.70%
测试集上召回率: 75.64%
测试集上F1值: 0.751659
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 20s - loss: 0.1107 - val_loss: 0.1350
测试集上正确率: 81.57%
测试集上准确率: 74.35%
测试集上召回率: 76.34%
测试集上F1值: 0.753319
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 20s - loss: 0.1098 - val_loss: 0.1357
测试集上正确率: 81.61%
测试集上准确率: 75.06%
测试集上召回率: 75.05%
测试集上F1值: 0.750570
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 20s - loss: 0.1093 - val_loss: 0.1351
测试集上正确率: 81.69%
测试集上准确率: 74.90%
测试集上召回率: 75.70%
测试集上F1值: 0.752987
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 21s - loss: 0.1085 - val_loss: 0.1350
测试集上正确率: 81.61%
测试集上准确率: 74.87%
测试集上召回率: 75.41%
测试集上F1值: 

363860/363860 [==============================] - 21s - loss: 0.1354 - val_loss: 0.1443
测试集上正确率: 79.74%
测试集上准确率: 71.39%
测试集上召回率: 74.41%
测试集上F1值: 0.728682
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 20s - loss: 0.1315 - val_loss: 0.1427
测试集上正确率: 79.74%
测试集上准确率: 72.20%
测试集上召回率: 72.49%
测试集上F1值: 0.723449
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 21s - loss: 0.1281 - val_loss: 0.1427
测试集上正确率: 79.72%
测试集上准确率: 70.27%
测试集上召回率: 77.19%
测试集上F1值: 0.735637
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 21s - loss: 0.1255 - val_loss: 0.1394
测试集上正确率: 80.52%
测试集上准确率: 72.44%
测试集上召回率: 75.41%
测试集上F1值: 0.738995
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 21s - loss: 0.1230 - val_loss: 0.1390
测试集上正确率: 80.61%
测试集上准确率: 72.82%
测试集上召回率: 74.93%
测试集上F1值: 

363860/363860 [==============================] - 21s - loss: 0.1003 - val_loss: 0.1342
测试集上正确率: 81.88%
测试集上准确率: 74.37%
测试集上召回率: 76.96%
测试集上F1值: 0.756433
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 20s - loss: 0.1000 - val_loss: 0.1355
测试集上正确率: 82.11%
测试集上准确率: 76.22%
测试集上召回率: 74.24%
测试集上F1值: 0.752168
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 21s - loss: 0.0998 - val_loss: 0.1337
测试集上正确率: 82.09%
测试集上准确率: 74.77%
测试集上召回率: 76.97%
测试集上F1值: 0.758568
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 20s - loss: 0.0996 - val_loss: 0.1339
测试集上正确率: 82.07%
测试集上准确率: 75.29%
测试集上召回率: 75.88%
测试集上F1值: 0.755821
Train on 363860 samples, validate on 40428 samples
Epoch 1/1
363860/363860 [==============================] - 21s - loss: 0.0994 - val_loss: 0.1343
测试集上正确率: 81.98%
测试集上准确率: 74.50%
测试集上召回率: 77.11%
测试集上F1值: 

In [15]:
from keras.models import Model
from keras.layers import Dense, Lambda, merge, BatchNormalization, Activation, Input, Merge
from keras import backend as K

def create_network(input_dim):
    # 网络初始化
    base_network = create_base_network(input_dim)
    
    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))
    
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
    
    model = Model(input=[input_a, input_b], output=distance)
    return model

def create_base_network(input_dim):
    
    input = Input(shape=(input_dim, ))
    dense1 = Dense(128)(input)
    bn1 = BatchNormalization()(dense1)
    relu1 = Activation('relu')(bn1)

    dense2 = Dense(128)(relu1)
    bn2 = BatchNormalization()(dense2)
    res2 = merge([relu1, bn2], mode='sum')
    relu2 = Activation('relu')(res2)    

    dense3 = Dense(128)(relu2)
    bn3 = BatchNormalization()(dense3)
    res3 = Merge(mode='sum')([relu2, bn3])
    relu3 = Activation('relu')(res3)   
    
    feats = merge([relu3, relu2, relu1], mode='concat')
    bn4 = BatchNormalization()(feats)

    model = Model(input=input, output=bn4)

    return model

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def compute_accuracy(predictions, labels):
    return labels[predictions.ravel() < 0.5].mean()

def compute_result(predictions, labels):
    result = {}
    predictions = predictions.ravel()
    pred_labels = (predictions < 0.5).astype('int')
    precision = labels[predictions < 0.5].mean()  # 准确率
    recall = pred_labels[labels == 1].mean()  # 召回率
    accuracy = (pred_labels == labels).astype('int').mean()  # 正确率
    f1 = 2*precision*recall/(precision+recall + 1e-8) # f1值
    result = {'accuracy':accuracy, 'precision':precision, 'recall':recall, 'f1':f1}
    return result